In [1]:
import firebase_kapp_api as f
import pandas as pd

In [161]:
csv = {
    'babymat': 'babymat.csv',
    'bakeartikler': 'bakeartikler.csv',
    'bakevarer': 'bakevarer.csv',
    'div_matprodukter': 'div_matprodukter.csv',
    'drikke': 'drikke.csv',
#     'fisk': 'fisk.csv',
#     'frukt_gront': 'frukt_gront.csv',
#     'helsekost': 'helsekost.csv',
#     'is_dessert': 'is_dessert.csv',
#     'meieri_ost': 'meieri_ost.csv',
#     'palegg_frokost': 'palegg_frokost.csv',
#     'snacks_godteri': 'snacks_godteri.csv',
}

In [162]:
find_duplicates_all()

------ Read Data
data/babymat.csv

------ Read Data
data/bakeartikler.csv

------ Read Data
data/bakevarer.csv

------ Read Data
data/div_matprodukter.csv

------ Read Data
data/drikke.csv

------ Find Duplicates


,category,sub_category,producer,product_type,product,kosher_type,kosher_stamp,comment
192,bakevarer,brød og rundstykker,Santa Maria,-,Mexican Pita,p,-,-
419,div matprodukter,Middagsprodukter,Santa Maria,-,Mexican Pita,p,-,-
67,bakeartikler,-,Tine,-,Nå Paibunn,p,-,-
435,div matprodukter,Middagsprodukter,Tine,-,Nå Paibunn,p,-,-
143,bakeartikler,-,Toro,-,Pannekaker,m,-,-
442,div matprodukter,Middagsprodukter,Toro,-,Pannekaker,m,-,-


In [143]:
# Read and load all files.
for k,v in csv.items():
    print('------------------------------------------------')
    df = read_data('data/' + v)
    insert_data(df)
    
count_products()

------------------------------------------------
------ Read Data
data/bakeartikler.csv

------ Batch Insert Data
+ bakeartikler_bakepulver, alle sorter
+ bakeartikler_gjær, tørr og fersk
+ bakeartikler_honning, alle sorter ren honning
+ bakeartikler_kakaopulver, alle sorter
+ bakeartikler_kokos, alle sorter
+ bakeartikler_mel, alle typer
+ bakeartikler_natron, alle typer
+ bakeartikler_nøtter, alle naturelle eller som kun er tilsatt salt og vegetabilsk olje
+ bakeartikler_potetmel, alle sorter
+ bakeartikler_presset sitron uten tilsetningsstoffer
+ bakeartikler_rosiner, alle sorter
+ bakeartikler_sukker, alle sorter uten fargetilsetning
+ dansukker_melis med sjokoladesmak
+ dansukker_fruktose
+ dansukker_gelesukker
+ dansukker_norsk lys sirup
+ dansukker_sukker, alle sorter uten farge- eller smakstilsetning
+ dansukker_syltesukker
+ eldorado_lys sirup
+ eldorado_mørk sirup
+ findus_smördeig butterdeig
+ freia_apsikat
+ freia_bakepulver
+ freia_bakekakao
+ freia_brown sugar sprinkles


In [174]:
# Read and load a single file.
df = read_data('data/' + csv['drikke'])
find_duplicates(df)
insert_data(df)
count_products()

------ Read Data
data/drikke.csv

------ Find Duplicates


,category,sub_category,producer,product_type,product,kosher_type,kosher_stamp,comment



------ Batch Insert Data
+ info_all saft, juice, brus og mineralvann som ikke inneholder druejus er kosher.
+ coca-cola_burn
+ coca-cola_alle andre coca-cola produkter
+ pepsi_alle produkter
+ ringnes_alle produkter
+ ringnes_mozell
+ ringnes_mozell light
+ ringnes_alle andre ringnes-produkter
+ schweppes_alle sorter
+ solo_alle produkter
+ ikea_cider apple and pear
+ ikea_drink lingon flader
+ ikea_blåbär
+ fun light_appelsin
+ fun light_bringebær
+ fun light_bringebær lime
+ fun light_cola
+ fun light_eple pære
+ fun light_familiy size fruktfest
+ fun light_family size bringebær & blåbær
+ fun light_family size sitrusfest
+ fun light_fruktfest
+ fun light_husholdningssaft
+ fun light_ice tea mango
+ fun light_ice tea mynte lime
+ fun light_icetea peach
+ fun light_jordbær kiwi
+ fun light_jungle fruit
+ fun light_lemon lime
+ fun light_lemonade
+ fun light_mango lime
+ fun light_red berry crush
+ fun light_strawberry passion
+ fun light_sweet lemon guarana
+ fun light_tropisk
+ l

In [112]:
def read_data(csv_file):
    """
    Also presents some information about the data.
    Returns: A Panda DataFrame.
    """
    print("------ Read Data")
    print(csv_file)
    df = pd.read_csv(csv_file)
    print()
    return df

In [155]:
def find_duplicates(df):
    """
    Find duplicates in the dataframe.
    """
    print('------ Find Duplicates')
    dups = df.duplicated(subset=['producer','product'], keep=False) 
    df_dups = df[dups].sort_values(by=['producer','product'])
    display(df_dups)
    print()

In [157]:
def find_duplicates_all():
    df = pd.DataFrame()
    for k,v in csv.items():
        df = pd.concat([df, read_data('data/' + v)], ignore_index=True)
    find_duplicates(df)

In [111]:
def insert_data(df):
    """
    Creates products in the Firebase database.
    Input: A DataFrame
    """
    print("------ Batch Insert Data")
    f.batch_create(df)
    print()

In [116]:
def count_products():
    """
    Count products in CSV files and in the database.
    """
    print("------ Count Products")
    csv_len_sum = 0
    for k,v in csv.items():
        df_len = len(pd.read_csv('data/' + v))
        csv_len_sum = csv_len_sum + df_len
        print("{}: {}".format(v, df_len))
    print("---")
    print("All files: {}".format(csv_len_sum))
    print("Database: {}".format(f.nofProducts()))
    print()

In [165]:
f.list_products(False)

id:-_

product:


id:-_akevitt
product:Akevitt

id:-_amaretto
product:Amaretto

id:-_arak pastis
product:Arak Pastis

id:-_bacardi
product:Bacardi

id:-_bailey’s original irish cream
product:Bailey’s Original Irish Cream

id:-_bols liqueur
product:Bols Liqueur

id:-_calvados
product:Calvados

id:-_campari bitter
product:Campari Bitter

id:-_cherry brandy
product:Cherry Brandy

id:-_cider
product:Cider

id:-_cointreau
product:Cointreau

id:-_drambuie
product:Drambuie

id:-_eggelikør
product:Eggelikør

id:-_gamla cabernet sauvignon
product:Gamla Cabernet Sauvignon

id:-_gamla nature brut
product:Gamla Nature Brut

id:-_gamla sangiovese
product:Gamla Sangiovese

id:-_gin
product:Gin

id:-_irish cream
product:Irish Cream

id:-_kahlua
product:Kahlua

id:-_kirschwasser
product:Kirschwasser

id:-_malibu
product:Malibu

id:-_mozartlikør
product:Mozartlikør

id:-_ouzo
product:Ouzo

id:-_pernod
product:Pernod

id:-_peter heering
product:Peter Heering

id:-_rom
product:Rom

id:-_sambuca
product:S

id:toro_kålstuing
product:Kålstuing

id:toro_kjøttdeigsaus
product:Kjøttdeigsaus

id:toro_kremet spinatsuppe
product:Kremet spinatsuppe

id:toro_kruspersille
product:Kruspersille

id:toro_krydder bearnaisesaus
product:Krydder Bearnaisesaus

id:toro_kyllingkrydder
product:Kyllingkrydder

id:toro_lammekjøttkrydder
product:Lammekjøttkrydder

id:toro_laurbærblad
product:Laurbærblad

id:toro_lett bearnaisesaus
product:Lett Bearnaisesaus

id:toro_løkpulver
product:Løkpulver

id:toro_majoran
product:Majoran

id:toro_malt allehånde
product:Malt allehånde

id:toro_malt cayennepepper spansk pepper
product:Malt Cayennepepper Spansk Pepper

id:toro_malt ingefær
product:Malt ingefær

id:toro_malt kardemomme
product:Malt kardemomme

id:toro_malt koriander
product:Malt koriander

id:toro_malt muskatblomme
product:Malt muskatblomme

id:toro_malt muskatnøtt
product:Malt muskatnøtt

id:toro_malt nellik
product:Malt nellik

id:toro_malt stjerneanis
product:Malt stjerneanis

id:toro_mexicansk chili-pu

In [167]:
# %%time
def get_docs():
    docs = f.products_ref().get()
    products = []
    for doc in docs:
        products.append(doc.to_dict())
    return products

p = get_docs()
p

[{'producer': '-',
  'kosher_type': 'p',
  'product_type': '-',
  'sub_category': 'Alkohol, Vin',
  'kosher_stamp': '-',
  'product': '\n',
  'category': 'drikke',
  'ts_created': DatetimeWithNanoseconds(2018, 12, 29, 19, 21, 56, 700000, tzinfo=<UTC>),
  'comment': '-'},
 {'product': 'Akevitt',
  'category': 'drikke',
  'ts_created': DatetimeWithNanoseconds(2018, 12, 29, 19, 21, 56, 912000, tzinfo=<UTC>),
  'comment': '-',
  'producer': '-',
  'kosher_type': 'p',
  'product_type': '-',
  'sub_category': 'Alkohol, Vin',
  'kosher_stamp': '-'},
 {'ts_created': DatetimeWithNanoseconds(2018, 12, 29, 19, 21, 57, 127000, tzinfo=<UTC>),
  'comment': '-',
  'producer': '-',
  'kosher_type': 'p',
  'product_type': '-',
  'sub_category': 'Alkohol, Vin',
  'kosher_stamp': '-',
  'product': 'Amaretto',
  'category': 'drikke'},
 {'producer': '-',
  'kosher_type': 'p',
  'product_type': '-',
  'sub_category': 'Alkohol, Vin',
  'kosher_stamp': '-',
  'product': 'Arak Pastis',
  'category': 'drikke',


In [15]:
docs = f.products_ref().get()

type(docs)

generator

### Misc

In [3]:
f.create({
    u'category': u'REMOVE',
    u'comment': u'',
    u'kosher_stamp': u'',
    u'kosher_type': u'p',
    u'producer': u'Toro',
    u'product': u'suppe',
    u'sub_category': u'',
})

x toro_suppe already exists


In [173]:
doc_ref1 = f.get_doc_ref(u'-_\n')

f.pp(doc_ref1)

Product (-_
) does not exists


In [9]:
# doc_ref1 = f.get_doc_ref(u'J7Rh7IcdEZ3Sxnb0N1yO')
# doc_ref2 = f.delete(f.get_doc_ref(u'J7Rh7IcdEZ3Sxnb0N1yO'))
doc_ref3 = f.get_doc_ref(u'213132')

f.update(doc_ref3, {u'comment':'Bye'})

Product (213132) does not exists
Product (213132) does not exists


In [10]:
doc_ref1 = f.get_doc_ref(u'J7Rh7IcdEZ3Sxnb0N1yO')
doc_ref2 = f.delete(f.get_doc_ref(u'nTk2VlJoA9MOXxZuMrBs'))

f.pp(doc_ref1)
display(f.isDeleted(doc_ref1))
display(f.exists(doc_ref1))

Product (J7Rh7IcdEZ3Sxnb0N1yO) does not exists
Product (nTk2VlJoA9MOXxZuMrBs) does not exists
Product (nTk2VlJoA9MOXxZuMrBs) does not exists


False

False

In [7]:
s = "Håndværker m/solsikkekjerner"
s
s.encode('unicode-escape')

'Håndværker m/solsikkekjerner'

b'Ha\\u030andv\\xe6rker m/solsikkekjerner'